In [1]:
import pandas as pd
import os
import numpy as np
import librosa
from sklearn.preprocessing import OneHotEncoder

In [2]:
paths=[]
labels=[]
for filename in os.listdir('./AudioWAV'):
    
    paths.append('./AudioWAV/' + filename)
    file = filename.split('.')[0]
   
    label = file.split('_')[2]
    if label == 'ANG':
        labels.append('angry.wav')
    elif label == 'DIS':
        labels.append('disgust.wav')
    elif label == 'FEA':
        labels.append('fear.wav')
    elif label == 'HAP':
        labels.append('happy.wav')
    elif label == 'NEU':
        labels.append('neutral.wav')
    elif label == 'SAD':
        labels.append('sad.wav')
        

df_cremad = pd.DataFrame({'speech':paths,'label':labels})
df_cremad.sample(5)

,speech,label
1044,./AudioWAV/1013_TSI_HAP_XX.wav,happy.wav
6617,./AudioWAV/1081_WSI_DIS_XX.wav,disgust.wav
1107,./AudioWAV/1014_MTI_FEA_XX.wav,fear.wav
2160,./AudioWAV/1027_IWW_DIS_XX.wav,disgust.wav
7216,./AudioWAV/1089_IEO_SAD_LO.wav,sad.wav


In [3]:
def MFCC(filename):
    y, sr = librosa.load(filename,duration=3,offset=0.5)
    return np.mean(librosa.feature.mfcc(y=y,sr=sr,n_mfcc=40).T,axis=0)

mfcc_cremad = df_cremad['speech'].apply(lambda x:MFCC(x))

In [4]:
X =[x for x in mfcc_cremad]
X =np.array(X)
X.shape
X =np.expand_dims(X,-1)
X.shape

(7442, 40, 1)

In [5]:
ohe=OneHotEncoder()
y = ohe.fit_transform(df_cremad[['label']] )
y = y.toarray()

In [6]:
X.shape, y.shape

((7442, 40, 1), (7442, 6))

In [7]:
df_cremad['label'].unique()

array(['angry.wav', 'disgust.wav', 'fear.wav', 'happy.wav', 'neutral.wav',
       'sad.wav'], dtype=object)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Definindo os modelos

class SimpleDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleDNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class LSTMAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        attn_weights = torch.softmax(self.attention(out), dim=1)
        out = torch.sum(attn_weights * out, dim=1)
        out = self.fc(out)
        return out

class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.fc_input_size = 32 * 1 * 1
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.dropout(x)
        x = x.view(-1, self.fc_input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


class CNNAttention(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNAttention, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.attention = nn.Linear(32, 1)
        self.fc = nn.Linear(32, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        attn_weights = self.softmax(self.attention(x.permute(0, 2, 1))).squeeze(-1)
        attn_weights = attn_weights.unsqueeze(-1)
        x = torch.sum(attn_weights * x, dim=2)
        x = self.fc(x)
        return x
    
# Construindo e treinando os modelos

input_size = X.shape[1:]
num_classes = y.shape[1]
hidden_size = 64
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Convertendo os dados para tensores PyTorch
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Definindo o tamanho do lote
batch_size = 32

# Criando conjuntos de dados PyTorch
dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Definindo tensor para o LSTM
X_tensorLSTM = X_tensor.permute(0, 2, 1)

# Criando conjuntos de dados PyTorch para LSTM
datasetLSTM = torch.utils.data.TensorDataset(X_tensorLSTM, y_tensor)
train_sizeLSTM = int(0.8 * len(datasetLSTM))
test_sizeLSTM = len(datasetLSTM) - train_sizeLSTM
train_datasetLSTM, test_datasetLSTM = torch.utils.data.random_split(datasetLSTM, [train_sizeLSTM, test_sizeLSTM])

# DataLoader para o LSTM
train_loaderLSTM = DataLoader(train_datasetLSTM, batch_size=batch_size, shuffle=True)
test_loaderLSTM = DataLoader(test_datasetLSTM, batch_size=batch_size, shuffle=False)

# Convertendo os dados para tensores PyTorch do CNN
X_tensorCNN = torch.tensor(X, dtype=torch.float32).to(device)

# Criando conjuntos de dados PyTorch do CNN
datasetCNN = torch.utils.data.TensorDataset(X_tensorCNN, y_tensor)
train_sizeCNN = int(0.8 * len(datasetCNN))
test_sizeCNN = len(datasetCNN) - train_sizeCNN
train_datasetCNN, test_datasetCNN = torch.utils.data.random_split(datasetCNN, [train_sizeCNN, test_sizeCNN])

# DataLoader para o CNN
train_loaderCNN = DataLoader(train_datasetCNN, batch_size=batch_size, shuffle=True)
test_loaderCNN = DataLoader(test_datasetCNN, batch_size=batch_size, shuffle=False)

# Função para treinamento
def train(model, train_loader, criterion, optimizer, num_epochs=500):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs.shape)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}')

In [9]:
# Standard Deep Neural Network
sdnn_model = SimpleDNN(input_size[0], num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(sdnn_model.parameters(), lr=0.0001)
train(sdnn_model, train_loader, criterion, optimizer)

Epoch [1/500], Loss: 8.9504, Accuracy: 0.1734
Epoch [2/500], Loss: 4.1523, Accuracy: 0.1759
Epoch [3/500], Loss: 2.5940, Accuracy: 0.1720
Epoch [4/500], Loss: 2.1372, Accuracy: 0.1786
Epoch [5/500], Loss: 1.9814, Accuracy: 0.1683
Epoch [6/500], Loss: 1.8996, Accuracy: 0.1729
Epoch [7/500], Loss: 1.8656, Accuracy: 0.1779
Epoch [8/500], Loss: 1.8484, Accuracy: 0.1702
Epoch [9/500], Loss: 1.8411, Accuracy: 0.1700
Epoch [10/500], Loss: 1.8158, Accuracy: 0.1777
Epoch [11/500], Loss: 1.8126, Accuracy: 0.1722
Epoch [12/500], Loss: 1.8044, Accuracy: 0.1710
Epoch [13/500], Loss: 1.8049, Accuracy: 0.1735
Epoch [14/500], Loss: 1.8062, Accuracy: 0.1688
Epoch [15/500], Loss: 1.8012, Accuracy: 0.1724
Epoch [16/500], Loss: 1.7989, Accuracy: 0.1732
Epoch [17/500], Loss: 1.7982, Accuracy: 0.1747
Epoch [18/500], Loss: 1.7943, Accuracy: 0.1720
Epoch [19/500], Loss: 1.7912, Accuracy: 0.1740
Epoch [20/500], Loss: 1.7930, Accuracy: 0.1734
Epoch [21/500], Loss: 1.7880, Accuracy: 0.1920
Epoch [22/500], Loss: 

In [10]:
# LSTM
lstm_model = LSTMModel(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.0001)
train(lstm_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7886, Accuracy: 0.1729
Epoch [2/500], Loss: 1.7664, Accuracy: 0.2508
Epoch [3/500], Loss: 1.7186, Accuracy: 0.3066
Epoch [4/500], Loss: 1.6540, Accuracy: 0.3074
Epoch [5/500], Loss: 1.6109, Accuracy: 0.3170
Epoch [6/500], Loss: 1.5894, Accuracy: 0.3313
Epoch [7/500], Loss: 1.5724, Accuracy: 0.3410
Epoch [8/500], Loss: 1.5580, Accuracy: 0.3393
Epoch [9/500], Loss: 1.5557, Accuracy: 0.3496
Epoch [10/500], Loss: 1.5476, Accuracy: 0.3538
Epoch [11/500], Loss: 1.5369, Accuracy: 0.3575
Epoch [12/500], Loss: 1.5360, Accuracy: 0.3597
Epoch [13/500], Loss: 1.5246, Accuracy: 0.3620
Epoch [14/500], Loss: 1.5231, Accuracy: 0.3659
Epoch [15/500], Loss: 1.5136, Accuracy: 0.3697
Epoch [16/500], Loss: 1.5068, Accuracy: 0.3665
Epoch [17/500], Loss: 1.4950, Accuracy: 0.3701
Epoch [18/500], Loss: 1.4991, Accuracy: 0.3739
Epoch [19/500], Loss: 1.4863, Accuracy: 0.3753
Epoch [20/500], Loss: 1.4849, Accuracy: 0.3798
Epoch [21/500], Loss: 1.4842, Accuracy: 0.3828
Epoch [22/500], Loss: 

In [11]:
# LSTM with Attention
lstm_atn_model = LSTMAttention(input_size[0], hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_atn_model.parameters(), lr=0.0001)
train(lstm_atn_model, train_loaderLSTM, criterion, optimizer)

Epoch [1/500], Loss: 1.7882, Accuracy: 0.2011
Epoch [2/500], Loss: 1.7621, Accuracy: 0.2930
Epoch [3/500], Loss: 1.7045, Accuracy: 0.2977
Epoch [4/500], Loss: 1.6441, Accuracy: 0.3029
Epoch [5/500], Loss: 1.6098, Accuracy: 0.3096
Epoch [6/500], Loss: 1.5951, Accuracy: 0.3203
Epoch [7/500], Loss: 1.5783, Accuracy: 0.3306
Epoch [8/500], Loss: 1.5630, Accuracy: 0.3397
Epoch [9/500], Loss: 1.5559, Accuracy: 0.3423
Epoch [10/500], Loss: 1.5472, Accuracy: 0.3492
Epoch [11/500], Loss: 1.5401, Accuracy: 0.3586
Epoch [12/500], Loss: 1.5320, Accuracy: 0.3605
Epoch [13/500], Loss: 1.5269, Accuracy: 0.3647
Epoch [14/500], Loss: 1.5190, Accuracy: 0.3674
Epoch [15/500], Loss: 1.5119, Accuracy: 0.3783
Epoch [16/500], Loss: 1.5030, Accuracy: 0.3758
Epoch [17/500], Loss: 1.5003, Accuracy: 0.3848
Epoch [18/500], Loss: 1.4962, Accuracy: 0.3825
Epoch [19/500], Loss: 1.4892, Accuracy: 0.3909
Epoch [20/500], Loss: 1.4836, Accuracy: 0.3887
Epoch [21/500], Loss: 1.4747, Accuracy: 0.3921
Epoch [22/500], Loss: 

In [12]:
# Convolutional Neural Network
cnn_model = CNNModel(input_size[0],num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.0001)
train(cnn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 1.8478, Accuracy: 0.1787
Epoch [2/500], Loss: 1.7549, Accuracy: 0.2308
Epoch [3/500], Loss: 1.7118, Accuracy: 0.2629
Epoch [4/500], Loss: 1.6637, Accuracy: 0.2935
Epoch [5/500], Loss: 1.6471, Accuracy: 0.3029
Epoch [6/500], Loss: 1.6288, Accuracy: 0.3099
Epoch [7/500], Loss: 1.6153, Accuracy: 0.3130
Epoch [8/500], Loss: 1.6027, Accuracy: 0.3198
Epoch [9/500], Loss: 1.5961, Accuracy: 0.3219
Epoch [10/500], Loss: 1.5928, Accuracy: 0.3262
Epoch [11/500], Loss: 1.5909, Accuracy: 0.3180
Epoch [12/500], Loss: 1.5850, Accuracy: 0.3215
Epoch [13/500], Loss: 1.5732, Accuracy: 0.3329
Epoch [14/500], Loss: 1.5763, Accuracy: 0.3339
Epoch [15/500], Loss: 1.5696, Accuracy: 0.3333
Epoch [16/500], Loss: 1.5646, Accuracy: 0.3402
Epoch [17/500], Loss: 1.5627, Accuracy: 0.3400
Epoch [18/500], Loss: 1.5645, Accuracy: 0.3365
Epoch [19/500], Loss: 1.5521, Accuracy: 0.3429
Epoch [20/500], Loss: 1.5515, Accuracy: 0.3457
Epoch [21/500], Loss: 1.5497, Accuracy: 0.3486
Epoch [22/500], Loss: 

In [13]:
# Convolutional Neural Network with Attention
cnn_atn_model = CNNAttention(input_size[0],num_classes).to(device)
optimizer = optim.Adam(cnn_atn_model.parameters(), lr=0.0001)
train(cnn_atn_model, train_loaderCNN, criterion, optimizer)

Epoch [1/500], Loss: 2.0271, Accuracy: 0.2019
Epoch [2/500], Loss: 1.7401, Accuracy: 0.2590
Epoch [3/500], Loss: 1.6848, Accuracy: 0.2928
Epoch [4/500], Loss: 1.6369, Accuracy: 0.3103
Epoch [5/500], Loss: 1.6080, Accuracy: 0.3225
Epoch [6/500], Loss: 1.5890, Accuracy: 0.3339
Epoch [7/500], Loss: 1.5713, Accuracy: 0.3459
Epoch [8/500], Loss: 1.5609, Accuracy: 0.3588
Epoch [9/500], Loss: 1.5560, Accuracy: 0.3526
Epoch [10/500], Loss: 1.5442, Accuracy: 0.3702
Epoch [11/500], Loss: 1.5367, Accuracy: 0.3649
Epoch [12/500], Loss: 1.5282, Accuracy: 0.3712
Epoch [13/500], Loss: 1.5212, Accuracy: 0.3771
Epoch [14/500], Loss: 1.5164, Accuracy: 0.3776
Epoch [15/500], Loss: 1.5080, Accuracy: 0.3780
Epoch [16/500], Loss: 1.5038, Accuracy: 0.3786
Epoch [17/500], Loss: 1.5080, Accuracy: 0.3848
Epoch [18/500], Loss: 1.5011, Accuracy: 0.3815
Epoch [19/500], Loss: 1.4942, Accuracy: 0.3869
Epoch [20/500], Loss: 1.4899, Accuracy: 0.3931
Epoch [21/500], Loss: 1.4877, Accuracy: 0.3889
Epoch [22/500], Loss: 

In [14]:
# Função para teste
def test(model, test_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, torch.max(labels, 1)[1])
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == torch.max(labels, 1)[1]).sum().item()

    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

In [15]:
print("Testing SimpleDNN:")
test(sdnn_model, test_loader, criterion)

Testing SimpleDNN:
Test Loss: 1.4331, Test Accuracy: 0.4486


In [16]:
print("\nTesting LSTMModel:")
test(lstm_model, test_loaderLSTM, criterion)



Testing LSTMModel:
Test Loss: 1.7635, Test Accuracy: 0.4312


In [17]:
print("\nTesting LSTMAttention:")
test(lstm_atn_model, test_loaderLSTM, criterion)


Testing LSTMAttention:
Test Loss: 1.6836, Test Accuracy: 0.4439


In [18]:
print("\nTesting CNNModel:")
test(cnn_model, test_loaderCNN, criterion)



Testing CNNModel:
Test Loss: 1.4547, Test Accuracy: 0.3976


In [19]:
print("\nTesting CNNAttention:")
test(cnn_atn_model, test_loaderCNN, criterion)


Testing CNNAttention:
Test Loss: 1.3652, Test Accuracy: 0.4520
